In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-12"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13505,2024-05-12,Japão B2.League,02:00,Shiga,Yamagata Wyverns,1.24,3.77,163.5,1.82,1.88,0.0,0.0,0.0,SOqTfu9U,0.806452,0.265252,0.549451,0.531915,0.071704,131.148,31.212993,0.237998,3.0,0.000000,0.000000,111.02,1.544,0.309887,0.200704,46.0,154.446,106.407120,0.688960,1.2,1.643168,1.369306,342.28,1.934,1.168623,0.604252,2.0,91,86,1.22,3.98,107.464,284.088,0.714164,0.022933,NaN,1.68,0.336,0.714286,0.806260,1.0,0.193740,2.17,0.434,6.382488,0.258968,0.4,0.141032
13506,2024-05-12,Nova Zelândia Nbl,03:30,Wellington Saints,Nelson Giants,1.13,5.46,190.5,1.87,1.79,0.0,0.0,0.0,xlxwQofl,0.884956,0.183150,0.534759,0.558659,0.068106,117.878,19.329938,0.163983,2.4,1.341641,0.559017,110.21,1.182,0.183902,0.155586,59.0,194.088,84.453355,0.435129,1.8,1.643168,0.912871,283.88,2.106,0.914210,0.434098,-3.0,107,94,1.03,3.02,0.000,0.000,0.929218,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13507,2024-05-12,Países Baixos Dbl,10:00,Groningen,BAL Weert,1.23,3.55,149.5,1.83,1.83,0.0,0.0,0.0,GIPUOdkl,0.813008,0.281690,0.546448,0.546448,0.094698,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,71,85,1.45,2.62,0.000,0.000,0.686397,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13508,2024-05-12,Nova Zelândia Nbl,00:00,Taranaki Airs,Southland Sharks,1.02,11.60,172.5,1.80,1.86,0.0,0.0,0.0,bcwZQRvr,0.980392,0.086207,0.555556,0.537634,0.066599,156.202,32.619393,0.208828,1.8,1.643168,0.912871,181.77,1.740,0.408595,0.234825,38.0,562.958,192.373487,0.341719,0.0,0.000000,NaN,787.50,8.422,3.591959,0.426497,-131.0,83,75,2.19,10.50,0.000,0.000,1.185609,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13509,2024-05-12,México Cibacopa,00:15,Rayos de Hermosillo,Venados de Mazatlan,1.38,2.88,176.5,1.80,1.86,0.0,0.0,0.0,YXUEZWDF,0.724638,0.347222,0.555556,0.537634,0.071860,153.778,50.221156,0.326582,1.8,1.643168,0.912871,110.92,1.612,0.525804,0.326181,37.0,211.322,73.852897,0.349480,1.2,1.643168,1.369306,147.05,2.466,0.883929,0.358446,-16.0,94,85,1.18,1.73,144.968,213.778,0.497963,0.023184,NaN,-0.38,-0.076,-5.000000,0.683551,0.4,-0.283551,0.91,0.182,10.329670,0.374794,0.5,0.125206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13609,2024-05-12,Argentina Liga A,20:00,Obera TC,San Lorenzo,1.49,2.50,151.5,1.84,1.86,0.0,0.0,0.0,rehOxTtE,0.671141,0.400000,0.543478,0.537634,0.071141,139.746,27.463190,0.196522,1.8,1.643168,0.912871,104.80,1.906,0.414765,0.217610,-9.0,129.154,23.349025,0.180784,1.8,1.643168,0.912871,165.76,1.670,0.375633,0.224930,25.0,80,74,1.31,2.24,148.492,175.426,0.357984,0.007644,NaN,2.53,0.506,0.968379,0.637588,0.6,-0.037588,3.10,0.620,2.419355,0.441502,0.4,-0.041502
13610,2024-05-12,Argentina Liga A,19:00,Obras,Gimnasia,1.58,2.29,168.5,1.87,1.79,0.0,0.0,0.0,hUsbWQJs,0.632911,0.436681,0.534759,0.558659,0.069593,167.754,54.511240,0.324947,1.8,1.643168,0.912871,225.68,1.820,0.555203,0.305056,13.0,128.782,46.170257,0.358515,1.8,1

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13527,13:00,Polônia Liga De Basquete,Dabrowa Gornicza,Trefl Sopot,1.75,Back Home
13536,14:00,Macedônia Do Norte Prva Liga,Pelister,Rabotnicki,1.63,Back Home
13537,13:00,Itália Série A2,Luiss Roma,Agrigento,1.60,Back Home
13538,20:15,Chile Lnb 2,Stadio Italiano,Arabe Valparaiso,1.57,Back Home
13540,20:30,Chile Lnb,Las Animas,CD Puerto Varas,2.68,Back Home
13568,21:00,Eua Nba,Minnesota Timberwolves,Denver Nuggets,1.68,Back Home
13588,09:00,Finlândia Korisliiga,Helsinki Seagulls,BC Nokia,1.57,Back Home
13589,07:15,Filipinas Copa Das Filipinas,Meralco Bolts,NLEX Road Warriors,1.63,Back Home
13603,02:15,Austrália Nbl1 Leste,Newcastle Falcons F,Manly W. F,2.48,Back Home
